In [1]:
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import warnings
import pandas as pd

In [2]:
cf_attrs = {
    'grid_mapping_name': 'lambert_conformal_conic',
    'standard_parallel': 38.5,
    'longitude_of_central_meridian': -97.5,
    'latitude_of_projection_origin': 38.5,
    'earth_radius': 6371229.0
}

point_lon = -72.717
point_lat = 39.969 

projection = ccrs.LambertConformal(central_longitude=262.5, 
                                   central_latitude=38.5, 
                                   standard_parallels=(38.5, 38.5),
                                    globe=ccrs.Globe(semimajor_axis=6371229,
                                                     semiminor_axis=6371229))

x, y = projection.transform_point(point_lon, point_lat, ccrs.PlateCarree())

warnings.filterwarnings('ignore') 

In [3]:
df_ws = pd.DataFrame(columns = ['Intial','Forecast','20m','40m','60m','80m','100m','120m','140m','160m','180m','200m'])

In [4]:
def get_wind(path):
    og = xr.open_dataset(path,engine='cfgrib',backend_kwargs=dict(filter_by_keys={'typeOfLevel': 'hybrid'}))
    ds = og.metpy.assign_crs(cf_attrs).metpy.assign_y_x()
    ds_2 = ds.sel(y=y,x=x,method='nearest')
    # ds_3 = ds_2.drop_vars(['clwmr','unknown','rwmr','snmr','grle'])
    ds_3 = ds_2.sel(hybrid=[1,2,3,4,5])
    ws = np.sqrt(np.add(ds_3['u'].to_numpy() ** 2,ds_3['v'].to_numpy()** 2))
    ds_3['ws'] = xr.DataArray(ws)
    ws_interp = np.interp(x=np.arange(20,220,20),xp=ds_3['gh'].to_numpy(),fp=ds_3['ws'].to_numpy())
    test_l = [str(ds_3.time.to_numpy())[:13],int(ds_3.step.to_numpy()) / 3.6000E+12]
    test_l2 = test_l + list(ws_interp)
    #print(test_l2)
    df_ws.loc[len(df_ws)] = test_l2
    #return test_l2

In [5]:
get_wind("/Users/nicolema/Downloads/hrrr.t00z.wrfnatf00.grib2")

In [6]:
get_wind("/Users/nicolema/Downloads/hrrr.t23z.wrfnatf00.grib2")

In [7]:
get_wind("/Users/nicolema/Downloads/hrrr.t23z.wrfnatf06.grib2")

In [8]:
df_ws

,Intial,Forecast,20m,40m,60m,80m,100m,120m,140m,160m,180m,200m
0,2021-05-14T00,0.0,3.560813,4.292114,4.338825,4.385536,4.270756,4.042478,3.814200,3.585921,3.343039,3.085041
1,2021-05-14T23,0.0,5.021877,6.324613,6.393904,6.463195,6.358967,6.126736,5.894504,5.662273,5.543627,5.566392
2,2021-05-14T23,6.0,7.138263,9.973102,10.391957,10.810811,11.113667,11.338983,11.564300,11.789616,11.813524,11.632470
